In [1]:
from ansatz_simulation_class import AnsatzSimulation
from genetic_quantum import QuanvLayer, QuantumModel, train_model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from patch_making import PatchExtraction, Quanv_2d
from torch import tensor
import matplotlib.pyplot as plt

### Instantiating genetic ansatz model

In [2]:
n_qubits = 4
patch_size = 2
num_classes = 2
input_size = 28
toy_chromosome = [['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], [None, 'ctrl_0', 'trgt_0', None], ['rz_gate', 'pauli_y', 'rx_gate', 'phase']]
quanv = QuanvLayer(n_qubits, patch_size, toy_chromosome, mode='2d')
toy_qhcnn = QuantumModel(n_qubits, num_classes, patch_size, input_size, toy_chromosome)
toy_qhcnn

TypeError: QuanvLayer.__init__() missing 1 required positional argument: 'parameters'

### Getting data and sampling it

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    PatchExtraction(patch_size)])
mnist_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)


In [4]:
len(mnist_dataset)

60000

In [4]:
selected_classes = [4, 9]
reduced_mnist = [mnist_dataset.__getitem__(index) for index in range(mnist_dataset.__len__()) if mnist_dataset.__getitem__(index)[1] in selected_classes]
len(reduced_mnist)

11791

In [6]:
mnist_dataset.__getitem__(4)

(tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.9098, 0.9922],
         [0.8235, 0.9882, 0.6588, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.9882],
         [0.7176, 0.0000, 0.3608, 0.9882],
         [0.0824, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.9922, 0.6902, 0.0000, 0.0314],
         [0

In [6]:
class_one = 0
class_two = 0
binary_dataset = []

for image in reduced_mnist:
    if image[1] == 4 and class_one < 50:
        binary_dataset.append(image)
        class_one +=1 
    elif image[1] == 9 and class_two < 50:
        binary_dataset.append(image)
        class_two += 1
        
    if class_one == 50 and class_two == 50:
        break

In [8]:
type(binary_dataset)

list

In [7]:
mnist_images, mnist_labels = zip(*binary_dataset)

In [9]:
tensor_mnist = torch.stack(mnist_images)

In [10]:
tensor_mnist.shape

torch.Size([100, 196, 4])

In [9]:
mnist_labels = [label%2 for label in mnist_labels]

In [10]:
mnist_labels = tensor(mnist_labels)

In [13]:
mnist_labels

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 0, 0])

In [11]:
tensor_mnist.squeeze(1).shape

torch.Size([100, 196, 4])

In [15]:
output = toy_qhcnn.forward(tensor_mnist)

Passing through quanvolution layer...
Image quantum processing time: 9.498604000080377
Quanvolution processing time: 9.578680300153792
Classical layers processing time: 0.03716449998319149


In [16]:
output

tensor([[-0.6441, -0.7448],
        [-0.6050, -0.7899],
        [-0.6508, -0.7374],
        [-0.6259, -0.7653],
        [-0.6629, -0.7243],
        [-0.6639, -0.7233],
        [-0.6302, -0.7604],
        [-0.6440, -0.7449],
        [-0.6436, -0.7453],
        [-0.6077, -0.7865],
        [-0.6893, -0.6970],
        [-0.6269, -0.7641],
        [-0.6145, -0.7785],
        [-0.6214, -0.7705],
        [-0.6537, -0.7343],
        [-0.6602, -0.7272],
        [-0.6450, -0.7437],
        [-0.6446, -0.7442],
        [-0.6548, -0.7330],
        [-0.6295, -0.7612],
        [-0.6319, -0.7584],
        [-0.6866, -0.6997],
        [-0.6434, -0.7455],
        [-0.5933, -0.8041],
        [-0.6237, -0.7677],
        [-0.6466, -0.7420],
        [-0.6499, -0.7383],
        [-0.6181, -0.7743],
        [-0.6484, -0.7400],
        [-0.6584, -0.7292],
        [-0.6630, -0.7242],
        [-0.6396, -0.7497],
        [-0.6162, -0.7766],
        [-0.5990, -0.7971],
        [-0.6222, -0.7695],
        [-0.6346, -0

### Training my model

In [12]:
from sampled_cv_dataset import SampledDataset4Training

quantum_processing = transforms.Compose([Quanv_2d(n_qubits, toy_chromosome)])
training_zeros_ones = SampledDataset4Training(binary_dataset, transform=quantum_processing)

batch_size = 4

training_loader = DataLoader(training_zeros_ones, batch_size, shuffle=True)

In [13]:
training_zeros_ones.__getitem__(0)['image'].shape

torch.Size([196, 4])

In [25]:
for i, sample in enumerate(training_zeros_ones):
    print(i, sample['image'].shape, sample['label'])

0 torch.Size([196, 4]) 4
1 torch.Size([196, 4]) 9
2 torch.Size([196, 4]) 4
3 torch.Size([196, 4]) 9
4 torch.Size([196, 4]) 4
5 torch.Size([196, 4]) 9
6 torch.Size([196, 4]) 4
7 torch.Size([196, 4]) 9
8 torch.Size([196, 4]) 9
9 torch.Size([196, 4]) 9
10 torch.Size([196, 4]) 9
11 torch.Size([196, 4]) 4
12 torch.Size([196, 4]) 9
13 torch.Size([196, 4]) 9
14 torch.Size([196, 4]) 4
15 torch.Size([196, 4]) 4
16 torch.Size([196, 4]) 4
17 torch.Size([196, 4]) 4
18 torch.Size([196, 4]) 9
19 torch.Size([196, 4]) 9
20 torch.Size([196, 4]) 4
21 torch.Size([196, 4]) 4
22 torch.Size([196, 4]) 9
23 torch.Size([196, 4]) 4
24 torch.Size([196, 4]) 9
25 torch.Size([196, 4]) 4
26 torch.Size([196, 4]) 4
27 torch.Size([196, 4]) 9
28 torch.Size([196, 4]) 4
29 torch.Size([196, 4]) 4
30 torch.Size([196, 4]) 4
31 torch.Size([196, 4]) 9
32 torch.Size([196, 4]) 9
33 torch.Size([196, 4]) 9
34 torch.Size([196, 4]) 4
35 torch.Size([196, 4]) 4
36 torch.Size([196, 4]) 4
37 torch.Size([196, 4]) 9
38 torch.Size([196, 4]

In [14]:
optimizer = torch.optim.SGD(toy_qhcnn.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
#loss = loss_fn(output, mnist_labels)
device = torch.device("cpu")

In [19]:
class ClassicalComponent(nn.Module):
    def __init__(self, num_classes, input_size):
        super().__init__()
        feature_size = input_size ** 2
        self.fc1 = nn.Linear(feature_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [24]:
simple_fc = ClassicalComponent(num_classes, input_size)
num_epochs = 4
train_model(simple_fc, training_loader, num_epochs)

AttributeError: 'str' object has no attribute 'flatten'

In [23]:
def train_model(model, train_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        
        for i, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad()  # Zero out previous gradients
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagate to calculate gradients
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            
            if i % 4 == 0:  # Print every 10 batches
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

In [20]:
train_model(toy_qhcnn, training_loader, loss_fn, optimizer, device, num_epochs=2)

Epoch 1/2:   0%|          | 0/25 [00:00<?, ?it/s]


Passing through quanvolution layer...


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [21]:
torch.cuda.is_available()

False

In [ ]:
def train_one_epoch(model, epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

TypeError: tensor or list of tensors expected, got <class 'str'>

### Testing tensor dimensions and quantum convolution

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 4
dummy_input = torch.zeros(input_size, input_size, 2).numpy()
patched_images = [image.extract_patches_2d(img, (patch_size, patch_size)) for img in dummy_input]
#dummy_output = quanv.forward(dummy_input)

tensor(patched_images).shape

torch.Size([4, 3, 2, 2])

In [ ]:
from sklearn.feature_extraction import image
from torch import tensor

input_size = 28

shape = (input_size, input_size)
dummy_input = np.zeros(shape, dtype=float)
patched_images = image.extract_patches_2d(dummy_input, (patch_size, patch_size)) 
#dummy_output = quanv.forward(dummy_input)
n_patches, patch_h, patch_w = tensor(patched_images).shape
patched_images = tensor(patched_images).contiguous().view(n_patches, patch_h*patch_w)
image._extract_patches()

In [ ]:
patched_images[0]

tensor([0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
patched_images.shape

torch.Size([729, 4])

In [ ]:
myAnsatz = AnsatzSimulation(n_qubits)
outputs = [myAnsatz.simulate_circuit(patch, 'rx', toy_chromosome) for patch in patched_images]

In [ ]:
tensor(outputs).shape

torch.Size([1458, 4])